### Objective

Build a machine translation model (Spainish --> English), using standard Encoder-Decoder architecture.

### SetUP

In [1]:
# connect to google drive
import os
import numpy as np

# mount google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# Check for GPU free memory
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
#!pip install psutil
#!pip install humanize
import psutil
import humanize
import GPUtil as GPU
GPUs = GPU.getGPUs()

# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print('='*40)
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
  print('='*40)
printm() 

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.9 GB  | Proc size: 156.3 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


In [0]:
# change root directory such that models are saved in google drive during training
root_dir = "/content/gdrive/My Drive/NLP/Git_MT"
os.chdir(root_dir)

### Imports

In [0]:
# insert the path for utility custom functions
import sys
sys.path.insert(0, os.path.join(root_dir, 'code_utils'))

# custom python functions and classes
from utils import read_corpus, batch_iter, pad_sents
from vocab import Vocab, VocabEntry

In [0]:
# basic packages
import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter, namedtuple
from docopt import docopt
from itertools import chain
import json
from typing import List, Tuple, Dict, Set, Union
from docopt import docopt

#pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence



#others
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
from tqdm import tqdm
from IPython.core.debugger import set_trace

In [0]:
# Logger

import logging
logger = logging.getLogger("Tensor")

file_handler = logging.FileHandler("Tensor_file.log")
stream_handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s:%(name)s:%(message)s')

file_handler.setFormatter(formatter)
stream_handler.setFormatter(formatter)

logger.addHandler(file_handler)

logger.addHandler(stream_handler)

logger.setLevel(logging.DEBUG)

# A helper function to check how tensor sizes change
def log_size(tsr: torch.Tensor, name: str):
    #cls = getclass()
    logger.debug(msg=f"{name} ==> size={tsr.shape}")

In [0]:
#! pip freeze > requirements.txt

### Load Data: EDA

In [9]:
# load data
train_es = 'en_es_data/train.es'
train_en = 'en_es_data/train.en'

dev_es = 'en_es_data/dev.es'
dev_en = 'en_es_data/dev.en'

test_es = 'en_es_data/test.es'
test_en = 'en_es_data/test.en'


train_data_src = read_corpus(train_es, source='src')
train_data_tgt = read_corpus(train_en, source='tgt')

dev_data_src = read_corpus(dev_es, source='src')
dev_data_tgt = read_corpus(dev_en, source='tgt')

test_data_src = read_corpus(test_es, source='src')
test_data_tgt = read_corpus(test_en, source='tgt')

train_data = list(zip(train_data_src,train_data_tgt))
dev_data = list(zip(dev_data_src,dev_data_tgt))
test_data = list(zip(test_data_src,test_data_tgt))

#
print("=="*40)
print("Number of examples in train: {}".format(len(train_data)))
print("Number of examples in valid: {}".format(len(dev_data)))
print("Number of examples in test: {}".format(len(test_data)))
#
print("=="*40)
print("Spanish --> English")
es, en = next(iter(dev_data))
print("Sp: {}".format(' '.join(es)))
print("En: {}".format(' '.join(en)))
print("=="*40)

# Build Vocab with train set

size = 50000
freq_cutoff= 2
vocab_file = 'en_es_data/vocab.json'

vocab = Vocab.build(train_data_src, train_data_tgt, size, freq_cutoff)
print('generated vocabulary, source %d words, target %d words' % (len(vocab.src), len(vocab.tgt)))

vocab.save(vocab_file)
print('vocabulary saved to %s' % vocab_file)

#
print("=="*40)
print('Note that the <s> and </s> tokens are added while vocab\
      initialization.\nThese tokens are also present in target\
      top frequent words. \nThat is why vocab size for target language is lesser by 2.')
print("=="*40)


# Check tokenization process
print("=="*40)
sents = [['I', 'asgjsssd', 'will', 'be', 'there', 'for', 'you.'], ['This', 'is', 'spartaaaaaaaa.']]
print("Tokenize:\n {} \n {}\n".format(' '.join(sents[0]), ' '.join(sents[1])))

print(vocab.tgt.to_input_tensor(sents, "cpu"))
#
print("=="*40)
print("Note that 3 and 0  are <unk> and <pad> tokens!")
print("=="*40)

Number of examples in train: 216617
Number of examples in valid: 851
Number of examples in test: 8064
Spanish --> English
Sp: El ao pasado proyect estas dos diapositivas para demostrar que la capa de hielo rtico, que durante los ltimos tres millones de aos ha sido del tamao de los 48 estados, se ha reducido en un 40 por ciento.
En: <s> Last year I showed these two slides so that  demonstrate that the arctic ice cap,  which for most of the last three million years  has been the size of the lower 48 states,  has shrunk by 40 percent. </s>
initialize source vocabulary ..
number of word types: 172418, number of word types w/ frequency >= 2: 80623
initialize target vocabulary ..
number of word types: 128873, number of word types w/ frequency >= 2: 64215
generated vocabulary, source 50004 words, target 50002 words
vocabulary saved to en_es_data/vocab.json
Note that the <s> and </s> tokens are added while vocab      initialization.
These tokens are also present in target      top frequent wor

### Encoder-decoder model parts

#### Model Embedding

In [0]:
class ModelEmbeddings(nn.Module): 
    """
    Class that converts input words to their embeddings.
    """
    def __init__(self, embed_size, vocab):
        """
        Init the Embedding layers.

        @param embed_size (int): Embedding size (dimensionality)
        @param vocab (Vocab): Vocabulary object containing src and tgt languages
                              See vocab.py in code_utils.
        """
        super(ModelEmbeddings, self).__init__()
        self.embed_size = embed_size

        # default values
        self.source = None
        self.target = None

        src_pad_token_idx = vocab.src['<pad>']
        tgt_pad_token_idx = vocab.tgt['<pad>']

        
        self.source = nn.Embedding(len(vocab.src), embed_size, padding_idx=src_pad_token_idx)
        self.target = nn.Embedding(len(vocab.tgt), embed_size, padding_idx=tgt_pad_token_idx)
      


#### Encoder 

In [0]:
## Encoder Block 
class Encoder(nn.Module):
  
  """ Class that encodes given sentences to finite dimensional vectors 
  """

  def __init__(self, embed_size, hidden_size, word_mat, n_layers=1, dropout_rate=0.2):
  
    """ Initialize the encoder block 
    @ param embed_size: embedding dimension
    @ param hidden_size: # of hidden units for LSTM
    @ word_mat:  ModelEmbeddings object
    @ param n_layers: # of LSTM layers
    @ dropout_rate: dropout for regularization
    """  

    super(Encoder, self).__init__()
    self.embed_size = embed_size
    self.hidden_size = hidden_size
    self.n_layers = n_layers
    self.dropout_rate = dropout_rate
    self.embed_mat = word_mat.source # to get embeddings of words from source language

    # Layers
    self.seq_src = nn.LSTM(embed_size, hidden_size, n_layers)
    self.dropout = nn.Dropout(dropout_rate)

  def forward(self, source_padded, source_lengths):
    
    """ Encodes the source sentence
        @param: source_padded ==> Tensor (T,b) #T = max src length, b= batch size 
        @param: sorce_lengths ==> List (b) : length of source sentences
        @return last_hidden ==> Tensor (n_layers*n_directions,b,h) # h = hidden_size
        @return last_cell => Tensor (n_layers*n_directions,b,h) # h = hidden_size
    """
    #set_trace()
    X = self.dropout(self.embed_mat(source_padded))
    X = pack_padded_sequence(X, source_lengths) # for faster computation
    _, (last_hidden, last_cell) = self.seq_src(X) 

    return last_hidden, last_cell


#### Decoder

In [0]:
# Decoder block
class Decoder(nn.Module):
  
  """ Class that decodes source sentences
  """

  def __init__(self, embed_size, hidden_size, word_mat, n_layers=1, dropout_rate =0.2):
    
    super(Decoder, self).__init__()
    
    self.embed_size = embed_size
    self.hidden_size = hidden_size
    self.n_layers = n_layers
    self.dropout_rate = dropout_rate
    self.embed_mat = word_mat.target # embedding layer
    self.vocab_size = self.embed_mat.weight.size(0)
    
    # Layers
    self.seq_tgt = nn.LSTM(embed_size, hidden_size, n_layers)
    self.dropout = nn.Dropout(dropout_rate)
    self.out = nn.Linear(hidden_size, self.vocab_size)

  def forward(self, Y_in, dec_state):
    
    """ Takes input of time step t and predicts conditional 
    probability of words at time t+1
    
    @ param: Y_embed: embedding for input token at time t
    @ param: decoder state: hidden unit activations at time t
    @ returns updated decoder state and next time step word predictions
    """
    
    Y_embed = self.embed_mat(Y_in) #(1,b,embed_size)
    d_out, dec_state = self.seq_tgt(Y_embed, dec_state)
    d_out = self.dropout(torch.squeeze(d_out,dim=0)) #(b,hidden_size)
    dec_pred = self.out(d_out)
    
    return dec_pred, dec_state

### Seq2Seq Model: Encoder-Decoder

In [0]:
class NMT(nn.Module):
  def __init__(self, vocab, encoder, decoder):
    super(NMT, self).__init__()
    
    self.encoder = encoder
    self.decoder = decoder
    self.vocab = vocab
    
    assert encoder.hidden_size == decoder.hidden_size, \
    "Hidden dimensions of encoder and decoder must be equal!"
    
    assert encoder.embed_size == decoder.embed_size, \
    "Embedding dimensions of encoder and decoder must be equal!"
    
    assert encoder.n_layers == decoder.n_layers, \
    "Encoder and decoder must have equal number of layers!"
    
    self.embed_size = encoder.embed_size
    self.hidden_size = encoder.hidden_size
    self.n_layers = encoder.n_layers
    
    
    # different layers  
    #self.model_embeddings = ModelEmbeddings(self.embed_size, vocab)
    
  def forward(self, source, target):
    # Compute sentence lengths
    source_lengths = [len(s) for s in source]
      
    # Convert list of lists into tensors
    source_padded = self.vocab.src.to_input_tensor(source, device=self.device)   # Tensor: (src_len, b)
    target_padded_raw = self.vocab.tgt.to_input_tensor(target, device=self.device)   # Tensor: (tgt_len, b)
    
    # Encode source padded --> decoder initial hidden state
    dec_hidden, dec_cell = self.encoder(source_padded, source_lengths)
    dec_state = (dec_hidden, dec_cell)
    
    # Chop of the <END> token for max length sentences.
    target_padded = target_padded_raw[:-1] #(tgt_len-1,b) ==> Tensor
    
    # Get target sentence length 
    tgt_len = target_padded.size(0)
    
    
    # split Y for input to each time step
    Y_splited = torch.split(target_padded,1, dim=0)
    
    # Genearte outputs with decoder
    outputs = [] # placeholder
    
    for i in range(tgt_len):
      
      Y_t = Y_splited[i] # Tensor (1,b)
      
      dec_pred, dec_state = self.decoder(Y_t, dec_state) #dec_pred ==> (b, tgt_vocab_len)
      
      outputs.append(dec_pred) 
    
    # stack  
    outputs = torch.stack(outputs) #(tgt_len,b,tgt_vocab_len)
    
    # apply softmax
    P = F.log_softmax(outputs, dim = -1) # P ==> (tgt_len,b,tgt_vocab_len)
    
    # Zero out, probabilities for which we have nothing in the target text
    target_masks = (target_padded_raw != self.vocab.tgt['<pad>']).float()
    
    # Compute log probability of generating true target words
    target_gold_words_log_prob = torch.gather(P, index=target_padded_raw[1:].unsqueeze(-1), dim=-1).squeeze(-1) * target_masks[1:]
    scores = target_gold_words_log_prob.sum(dim=0)
      
    return scores   
  
  @staticmethod
  def load(model_path: str):
      """ Load the model from a file.
      @param model_path (str): path to model
      """
      params = torch.load(model_path, map_location=lambda storage, loc: storage)
      args = params['args']
      model = NMT_base(vocab=params['vocab'], **args)
      model.load_state_dict(params['state_dict'])

      return model
    
    
  def save(self, path: str):
      """ Save the odel to a file.
      @param path (str): path to the model
      """
      print('save model parameters to [%s]' % path, file=sys.stderr)

      params = {
          'args': dict(embed_size=self.encoder.embed_size, hidden_size=self.hidden_size, n_layers=self.n_layers),
          'vocab': self.vocab,
          'state_dict': self.state_dict()
      }

      torch.save(params, path)
    
  @property
  def device(self) -> torch.device:
      """ Determine which device to place the Tensors upon, CPU or GPU.
      """
      return self.encoder.embed_mat.weight.device
    

### Build Model

In [0]:
# Hyperparameters

embed_size = 256
hidden_size = 512
n_layers = 1
enc_dropout = 0.2
dec_dropout = 0.2

# Build model
# Create model embedding object
word_mat = ModelEmbeddings(embed_size, vocab)
# Create Encoder
enc = Encoder(embed_size, hidden_size, word_mat, n_layers, enc_dropout)
# Create Decoder
dec = Decoder(embed_size, hidden_size, word_mat, n_layers, dec_dropout)
# Build Seq2Seq Model
model = NMT(vocab,enc, dec)

#### Initialize parameters and optimizer

In [42]:
uniform_init = 0.1

def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -uniform_init, uniform_init)
        
model.apply(init_weights)

# Count total parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

# Use Adam Optimizaer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# transfer model to cuda if available
device = torch.device("cuda:0" if torch.cuda.device_count()>0 else "cpu")
print('use device: %s' % device)
model = model.to(device)

The model has 54,406,482 trainable parameters
use device: cuda:0


## Training

#### Perplexity (PPL)

In [0]:
## Compute Perplexity to keep track of training

def evaluate_ppl(model, dev_data, batch_size=32):
    """ Evaluate perplexity on dev sentences
    @param model (NMT): NMT Model
    @param dev_data (list of (src_sent, tgt_sent)): list of tuples containing source and target sentence
    @param batch_size (batch size)
    @returns ppl (perplixty on dev sentences)
    """
    was_training = model.training
    model.eval()

    cum_loss = 0.
    cum_tgt_words = 0.

    # no_grad() signals backend to throw away all gradients
    with torch.no_grad():
        for src_sents, tgt_sents in batch_iter(dev_data, batch_size):
            loss = -model(src_sents, tgt_sents).sum()

            cum_loss += loss.item()
            tgt_word_num_to_predict = sum(len(s[1:]) for s in tgt_sents)  # omitting leading `<s>`
            cum_tgt_words += tgt_word_num_to_predict

        ppl = np.exp(cum_loss / cum_tgt_words)

    if was_training:
        model.train()

    return ppl

#### Model training function

In [0]:
######## Train Model ########

model_save_path = 'NMT_LSTM_seq2seq_one_layer'

##
def train_model(model, optimizer, clip_grad =5.0, max_epoch =30, max_patience = 3, max_trial = 3, lr_decay = 0.5, train_batch_size = 128, log_every = 100, valid_niter = 1000):
  
  
  print('Training begins...')
  ## Temp variables
  num_trial = 0
  train_iter = patience = cum_loss = report_loss = cum_tgt_words = report_tgt_words = 0
  cum_examples = report_examples  = valid_num = 0
  hist_valid_scores = []
  train_time = begin_time = time.time()
  
  # put the model in training mode
  model.train()
  
  
  # iterate over the epochs
  for epoch in range(max_epoch):
    for src_sents, tgt_sents in batch_iter(train_data, batch_size=train_batch_size, shuffle=True):
        
        train_iter += 1
        optimizer.zero_grad()
        batch_size = len(src_sents)
        
        example_losses = -model(src_sents, tgt_sents)
        batch_loss = example_losses.sum()
        loss = batch_loss/batch_size
        loss.backward() # autograd
        
        # Clip gradient
        grad_norn = torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)
        optimizer.step() # update parameters
        
        batch_losses_val = batch_loss.item()
        report_loss += batch_losses_val
        cum_loss += batch_losses_val
        
        tgt_words_num_to_predict = sum(len(s[1:]) for s in tgt_sents)  # omitting leading `<s>`
        report_tgt_words += tgt_words_num_to_predict
        cum_tgt_words += tgt_words_num_to_predict
        report_examples += batch_size
        cum_examples += batch_size
        
        # print interim report about training
        
        if train_iter % log_every == 0:
            #set_trace()
            print('| Epoch %d, Iter %d| Avg Loss = %.2f| Avg. ppl = %.2f| Speed %.2f words/sec| Time %.2f min|' % (epoch+1, train_iter, report_loss / report_examples, math.exp(report_loss / report_tgt_words),
                                                                                     report_tgt_words / (time.time() - train_time), (time.time() - begin_time)/60.0))

            train_time = time.time()
            report_loss = report_tgt_words = report_examples = 0.
        
        # validation
        if train_iter % valid_niter == 0:
            
            print('| <Train Summary> | Epoch %d, Iter %d| Cum. loss = %.2f| Cum. ppl = %.2f|' % (epoch+1, train_iter, cum_loss / cum_examples, np.exp(cum_loss / cum_tgt_words)))

            cum_loss = cum_examples = cum_tgt_words = 0.
            valid_num += 1

            print('Report on validation set:', file=sys.stderr)

            # compute dev. ppl and bleu
            dev_ppl = evaluate_ppl(model, dev_data, batch_size=128)   # dev batch size can be a bit larger
            valid_metric = -dev_ppl

            print('Validation:  Dev. ppl = %f' % (dev_ppl), file=sys.stderr)

            
            # learning rate scheduling
            
            is_better = (len(hist_valid_scores) == 0 or valid_metric > max(hist_valid_scores))
            hist_valid_scores.append(valid_metric)

            if is_better:
                patience = 0
                print('Save currently the best model to [%s]' % model_save_path, file=sys.stderr)
                model.save(model_save_path)

                # also save the optimizers' state
                torch.save(optimizer.state_dict(), model_save_path + '.optim')
                
            elif patience < int(max_patience):
                patience += 1
                print('Hit patience %d' % patience, file=sys.stderr)

                if patience == int(max_patience):
                    num_trial += 1
                    print('Hit #%d trial' % num_trial, file=sys.stderr)
                    
                    if num_trial == int(max_trial):
                        print('early stop!', file=sys.stderr)
                        return

                    # decay lr, and restore from previously best checkpoint
                    lr = optimizer.param_groups[0]['lr'] * float(lr_decay)
                    print('load previously best model and decay learning rate to %f' % lr, file=sys.stderr)

                    # load model
                    params = torch.load(model_save_path, map_location=lambda storage, loc: storage)
                    model.load_state_dict(params['state_dict'])
                    model = model.to(device)

                    print('restore parameters of the optimizers', file=sys.stderr)
                    optimizer.load_state_dict(torch.load(model_save_path + '.optim'))

                    # set new lr
                    for param_group in optimizer.param_groups:
                        param_group['lr'] = lr

                    # reset patience
                    patience = 0

            if epoch +1 == int(max_epoch):
                print('Training stopped <-> Reached maximum number of epochs!', file=sys.stderr)
                return

In [45]:
# train parameters
max_epoch =30
train_batch_size = 128

# train the model
train_model(model, optimizer, max_epoch =max_epoch, train_batch_size = train_batch_size)

Training begins...
| Epoch 1, Iter 100| Avg Loss = 125.55| Avg. ppl = 1234.10| Speed 3880.12 words/sec| Time 0.97 min|
| Epoch 1, Iter 200| Avg Loss = 110.54| Avg. ppl = 506.61| Speed 3871.41 words/sec| Time 1.95 min|
| Epoch 1, Iter 300| Avg Loss = 104.44| Avg. ppl = 362.79| Speed 3818.79 words/sec| Time 2.94 min|
| Epoch 1, Iter 400| Avg Loss = 101.38| Avg. ppl = 305.26| Speed 3865.46 words/sec| Time 3.92 min|
| Epoch 1, Iter 500| Avg Loss = 97.59| Avg. ppl = 263.03| Speed 3797.05 words/sec| Time 4.90 min|
| Epoch 1, Iter 600| Avg Loss = 97.35| Avg. ppl = 237.80| Speed 3837.94 words/sec| Time 5.89 min|
| Epoch 1, Iter 700| Avg Loss = 95.62| Avg. ppl = 218.89| Speed 3817.28 words/sec| Time 6.88 min|
| Epoch 1, Iter 800| Avg Loss = 92.71| Avg. ppl = 195.04| Speed 3801.30 words/sec| Time 7.87 min|
| Epoch 1, Iter 900| Avg Loss = 91.98| Avg. ppl = 180.72| Speed 3830.55 words/sec| Time 8.85 min|
| Epoch 1, Iter 1000| Avg Loss = 89.82| Avg. ppl = 167.01| Speed 3805.90 words/sec| Time 9.84 

Report on validation set:
Validation:  Dev. ppl = 172.725205
Save currently the best model to [NMT_LSTM_seq2seq_one_layer]
save model parameters to [NMT_LSTM_seq2seq_one_layer]


| Epoch 1, Iter 1100| Avg Loss = 89.57| Avg. ppl = 156.58| Speed 3344.71 words/sec| Time 10.97 min|
| Epoch 1, Iter 1200| Avg Loss = 88.29| Avg. ppl = 147.12| Speed 3789.53 words/sec| Time 11.96 min|
| Epoch 1, Iter 1300| Avg Loss = 86.49| Avg. ppl = 137.82| Speed 3776.01 words/sec| Time 12.95 min|
| Epoch 1, Iter 1400| Avg Loss = 86.73| Avg. ppl = 132.59| Speed 3821.52 words/sec| Time 13.95 min|
| Epoch 1, Iter 1500| Avg Loss = 85.06| Avg. ppl = 123.91| Speed 3783.49 words/sec| Time 14.94 min|
| Epoch 1, Iter 1600| Avg Loss = 83.95| Avg. ppl = 119.23| Speed 3786.74 words/sec| Time 15.93 min|
| Epoch 2, Iter 1700| Avg Loss = 84.20| Avg. ppl = 113.32| Speed 3817.81 words/sec| Time 16.92 min|
| Epoch 2, Iter 1800| Avg Loss = 80.31| Avg. ppl = 92.75| Speed 3802.06 words/sec| Time 17.91 min|
| Epoch 2, Iter 1900| Avg Loss = 79.83| Avg. ppl = 90.46| Speed 3778.86 words/sec| Time 18.91 min|
| Epoch 2, Iter 2000| Avg Loss = 78.82| Avg. ppl = 87.68| Speed 3794.05 words/sec| Time 19.90 min|
| <

Report on validation set:
Validation:  Dev. ppl = 115.323163
Save currently the best model to [NMT_LSTM_seq2seq_one_layer]
save model parameters to [NMT_LSTM_seq2seq_one_layer]


| Epoch 2, Iter 2100| Avg Loss = 78.50| Avg. ppl = 85.78| Speed 3509.21 words/sec| Time 20.98 min|
| Epoch 2, Iter 2200| Avg Loss = 77.82| Avg. ppl = 84.01| Speed 3776.84 words/sec| Time 21.97 min|
| Epoch 2, Iter 2300| Avg Loss = 77.73| Avg. ppl = 81.99| Speed 3779.82 words/sec| Time 22.96 min|
| Epoch 2, Iter 2400| Avg Loss = 77.91| Avg. ppl = 80.33| Speed 3806.67 words/sec| Time 23.96 min|
| Epoch 2, Iter 2500| Avg Loss = 77.22| Avg. ppl = 79.77| Speed 3793.40 words/sec| Time 24.95 min|
| Epoch 2, Iter 2600| Avg Loss = 75.93| Avg. ppl = 77.60| Speed 3790.54 words/sec| Time 25.93 min|
| Epoch 2, Iter 2700| Avg Loss = 77.82| Avg. ppl = 76.69| Speed 3908.90 words/sec| Time 26.91 min|
| Epoch 2, Iter 2800| Avg Loss = 76.23| Avg. ppl = 74.13| Speed 3786.46 words/sec| Time 27.91 min|
| Epoch 2, Iter 2900| Avg Loss = 75.55| Avg. ppl = 73.04| Speed 3775.67 words/sec| Time 28.90 min|
| Epoch 2, Iter 3000| Avg Loss = 75.78| Avg. ppl = 71.36| Speed 3792.27 words/sec| Time 29.90 min|
| <Train S

...

Report on validation set:
Validation:  Dev. ppl = 59.717359
Hit patience 1


| Epoch 11, Iter 17100| Avg Loss = 44.43| Avg. ppl = 12.25| Speed 3737.12 words/sec| Time 170.94 min|
| Epoch 11, Iter 17200| Avg Loss = 44.81| Avg. ppl = 12.39| Speed 3847.83 words/sec| Time 171.93 min|
| Epoch 11, Iter 17300| Avg Loss = 44.03| Avg. ppl = 12.22| Speed 3747.72 words/sec| Time 172.93 min|
| Epoch 11, Iter 17400| Avg Loss = 44.69| Avg. ppl = 12.43| Speed 3799.12 words/sec| Time 173.93 min|
| Epoch 11, Iter 17500| Avg Loss = 44.36| Avg. ppl = 12.40| Speed 3794.83 words/sec| Time 174.92 min|
| Epoch 11, Iter 17600| Avg Loss = 44.77| Avg. ppl = 12.56| Speed 3759.54 words/sec| Time 175.92 min|
| Epoch 11, Iter 17700| Avg Loss = 44.71| Avg. ppl = 12.50| Speed 3792.94 words/sec| Time 176.92 min|
| Epoch 11, Iter 17800| Avg Loss = 44.83| Avg. ppl = 12.54| Speed 3806.05 words/sec| Time 177.91 min|
| Epoch 11, Iter 17900| Avg Loss = 44.76| Avg. ppl = 12.54| Speed 3751.90 words/sec| Time 178.92 min|
| Epoch 11, Iter 18000| Avg Loss = 44.57| Avg. ppl = 12.58| Speed 3768.41 words/se

Report on validation set:
Validation:  Dev. ppl = 59.130569
Hit patience 2


| Epoch 11, Iter 18100| Avg Loss = 44.53| Avg. ppl = 12.67| Speed 3685.93 words/sec| Time 180.93 min|
| Epoch 11, Iter 18200| Avg Loss = 45.55| Avg. ppl = 12.85| Speed 3831.64 words/sec| Time 181.92 min|
| Epoch 11, Iter 18300| Avg Loss = 44.75| Avg. ppl = 12.71| Speed 3754.95 words/sec| Time 182.92 min|
| Epoch 11, Iter 18400| Avg Loss = 44.67| Avg. ppl = 12.73| Speed 3746.10 words/sec| Time 183.92 min|
| Epoch 11, Iter 18500| Avg Loss = 45.02| Avg. ppl = 12.80| Speed 3779.21 words/sec| Time 184.92 min|
| Epoch 11, Iter 18600| Avg Loss = 45.00| Avg. ppl = 12.90| Speed 3801.50 words/sec| Time 185.91 min|
| Epoch 12, Iter 18700| Avg Loss = 43.26| Avg. ppl = 11.48| Speed 3786.23 words/sec| Time 186.90 min|
| Epoch 12, Iter 18800| Avg Loss = 42.50| Avg. ppl = 11.16| Speed 3770.47 words/sec| Time 187.89 min|
| Epoch 12, Iter 18900| Avg Loss = 42.38| Avg. ppl = 11.17| Speed 3784.07 words/sec| Time 188.88 min|
| Epoch 12, Iter 19000| Avg Loss = 43.19| Avg. ppl = 11.45| Speed 3797.08 words/se

Report on validation set:
Validation:  Dev. ppl = 60.063730
Hit patience 3
Hit #3 trial
early stop!


### Evaluation

#### Beam Search

In [0]:
# Create Hypothesis tuple for beam search
Hypothesis = namedtuple('Hypothesis', ['value', 'score'])

In [0]:
def beam_search(model, src_sent: List[str], beam_size: int=5, max_decoding_time_step: int=70) -> List[Hypothesis]:
    """ Given a single source sentence, perform beam search, yielding translations in the target language.
    @param src_sent (List[str]): a single source sentence (words)
    @param beam_size (int): beam size
    @param max_decoding_time_step (int): maximum number of time steps to unroll the decoding RNN
    @returns hypotheses (List[Hypothesis]): a list of hypothesis, each hypothesis has two fields:
            value: List[str]: the decoded target sentence, represented as a list of words
            score: float: the log-likelihood of the target sentence
    """
    src_sents_var = model.vocab.src.to_input_tensor([src_sent], model.device)

    dec_init_vec = model.encoder(src_sents_var, [len(src_sent)])
    #src_encodings_att_linear = self.att_projection(src_encodings)

    h_tm1 = dec_init_vec
    #att_tm1 = torch.zeros(1, self.hidden_size, device=self.device)

    eos_id = model.vocab.tgt['</s>']

    hypotheses = [['<s>']]
    hyp_scores = torch.zeros(len(hypotheses), dtype=torch.float, device=model.device)
    completed_hypotheses = []

    t = 0
    while len(completed_hypotheses) < beam_size and t < max_decoding_time_step:
        t += 1
        hyp_num = len(hypotheses)
        #set_trace()
        y_tm1 = torch.tensor([model.vocab.tgt[hyp[-1]] for hyp in hypotheses], dtype=torch.long, device=model.device)
        
        #y_tm1 = y_tm1.transpose(0,1)
        #set_trace()
        #y_t_embed = model.model_embeddings.target(y_tm1)
        y_tm1 = torch.unsqueeze(y_tm1,dim=0)
        #y_t_embed = torch.unsqueeze(y_t_embed, dim=0)
        
        w_t, (h_t, cell_t) = model.decoder(y_tm1, dec_init_vec) # w_t --> (1, batch, hidden_size)
        
        #w_t = w_t.squeeze(dim=0)
        
        #output_t = model.target_vocab_projection(w_t)
        
        log_p_t = F.log_softmax(w_t, dim=-1)
        #set_trace()

        live_hyp_num = beam_size - len(completed_hypotheses)
        contiuating_hyp_scores = (hyp_scores.unsqueeze(1).expand_as(log_p_t) + log_p_t).view(-1)
        top_cand_hyp_scores, top_cand_hyp_pos = torch.topk(contiuating_hyp_scores, k=live_hyp_num)

        prev_hyp_ids = top_cand_hyp_pos / len(model.vocab.tgt)
        hyp_word_ids = top_cand_hyp_pos % len(model.vocab.tgt)

        new_hypotheses = []
        live_hyp_ids = []
        new_hyp_scores = []

        for prev_hyp_id, hyp_word_id, cand_new_hyp_score in zip(prev_hyp_ids, hyp_word_ids, top_cand_hyp_scores):
            prev_hyp_id = prev_hyp_id.item()
            hyp_word_id = hyp_word_id.item()
            cand_new_hyp_score = cand_new_hyp_score.item()

            hyp_word = model.vocab.tgt.id2word[hyp_word_id]
            new_hyp_sent = hypotheses[prev_hyp_id] + [hyp_word]
            if hyp_word == '</s>':
                completed_hypotheses.append(Hypothesis(value=new_hyp_sent[1:-1],
                                                       score=cand_new_hyp_score))
            else:
                new_hypotheses.append(new_hyp_sent)
                live_hyp_ids.append(prev_hyp_id)
                new_hyp_scores.append(cand_new_hyp_score)

        if len(completed_hypotheses) == beam_size:
            break
        #set_trace()
        live_hyp_ids = torch.tensor(live_hyp_ids, dtype=torch.long, device=model.device)
        dec_init_vec = (h_t[:,live_hyp_ids], cell_t[:,live_hyp_ids])
        #att_tm1 = att_t[live_hyp_ids]
        

        hypotheses = new_hypotheses
        hyp_scores = torch.tensor(new_hyp_scores, dtype=torch.float, device=model.device)
        #set_trace()

    if len(completed_hypotheses) == 0:
        completed_hypotheses.append(Hypothesis(value=hypotheses[0][1:],
                                               score=hyp_scores[0].item()))

    completed_hypotheses.sort(key=lambda hyp: hyp.score, reverse=True)

    return completed_hypotheses  

### BLEU
Evaluate Model BLEU score

In [0]:
def test_beam_search(model: NMT, test_data_src: List[List[str]], beam_size: int, max_decoding_time_step: int) -> List[List[Hypothesis]]:
    """ Run beam search to construct hypotheses for a list of src-language sentences.
    @param model (NMT): NMT Model
    @param test_data_src (List[List[str]]): List of sentences (words) in source language, from test set.
    @param beam_size (int): beam_size (# of hypotheses to hold for a translation at every step)
    @param max_decoding_time_step (int): maximum sentence length that Beam search can produce
    @returns hypotheses (List[List[Hypothesis]]): List of Hypothesis translations for every source sentence.
    """
    was_training = model.training
    model.eval()

    hypotheses = []
    with torch.no_grad():
        for src_sent in tqdm(test_data_src, desc='Decoding', file=sys.stdout):
            example_hyps = beam_search(model, src_sent, beam_size=beam_size, max_decoding_time_step=max_decoding_time_step)

            hypotheses.append(example_hyps)

    if was_training: model.train(was_training)

    return hypotheses
  

In [0]:
def compute_corpus_level_bleu_score(references: List[List[str]], hypotheses: List[Hypothesis]) -> float:
    """ Given decoding results and reference sentences, compute corpus-level BLEU score.
    @param references (List[List[str]]): a list of gold-standard reference target sentences
    @param hypotheses (List[Hypothesis]): a list of hypotheses, one for each reference
    @returns bleu_score: corpus-level BLEU score
    """
    if references[0][0] == '<s>':
        references = [ref[1:-1] for ref in references]
    bleu_score = corpus_bleu([[ref] for ref in references],
                             [hyp.value for hyp in hypotheses])
    return bleu_score

In [0]:
def decode(model):
    """ Performs decoding on a test set, and save the best-scoring decoding results.
    If the target gold-standard sentences are given, the function also computes
    corpus-level BLEU score.
    @param args (Dict): args from cmd line
    """

    print("load test source sentences", file=sys.stderr)
    test_data_src = read_corpus(test_es, source='src')
    
    
    print("load test target sentences", file=sys.stderr)
    test_data_tgt = read_corpus(test_en, source='tgt')

    #print("load trained model", file=sys.stderr)
    #model = NMT_base.load(model_save_path)

    #device = torch.device("cuda:0" if torch.cuda.device_count()>0 else "cpu")
    if torch.cuda.is_available():
        print("Transfer to cuda!!")
        model = model.to(torch.device("cuda:0"))

    hypotheses = test_beam_search(model, test_data_src,
                             beam_size=5,
                             max_decoding_time_step=70)


    top_hypotheses = [hyps[0] for hyps in hypotheses]
    bleu_score = compute_corpus_level_bleu_score(test_data_tgt, top_hypotheses)
    print('Corpus BLEU: {}'.format(bleu_score * 100), file=sys.stderr)

    with open('test_output.txt', 'w') as f:
        for src_sent, hyps in zip(test_data_src, hypotheses):
            top_hyp = hyps[0]
            hyp_sent = ' '.join(top_hyp.value)
            f.write(hyp_sent + '\n')

In [51]:
decode(model)

Transfer to cuda!!
Decoding:   0%|          | 0/8064 [00:00<?, ?it/s]

load test source sentences
load test target sentences


Decoding: 100%|██████████| 8064/8064 [07:01<00:00, 19.15it/s]


Corpus BLEU: 8.029102353804074


#### Inference

In [52]:
####
src_sent =['También', 'tenemos', 'que', 'tener', 'cuidado', 'con', 'el', 'hielo', 'se', 'resbala', 'fácilmente', 'en', 'él.']
en_ref = ['We', 'also', 'have', 'to', 'be', 'careful', 'with', 'the', 'ice,','it', 'slides', 'easily', 'on', 'it.']
en_hat = beam_search(model,src_sent,5,70)
print("=="*40)
print("Model Translation:\n")
print('{}'.format(' '.join(en_hat[0].value)))
print("\n")
print("Human Reference:\n")
print('{}'.format(' '.join(en_ref)))
print("=="*40)

Model Translation:

<unk> we have to be able to cross the ice course of <unk> <unk>


Human Reference:

We also have to be careful with the ice, it slides easily on it.
